In [2]:
import numpy as np


# Bayes Model From Scratch

In [1]:
class Bayes():
    def __init__(self):

        self.features = list()
        self.prior = {}
        self.likelihood = {}
        self.pred_prior = {}
        self.train_size = int
        self.num_feats = int
        self.X_train = np.array
        self.y_train = np.array

    def findClassPrior(self):
        """ P(c) - Prior class probability P(c) = count(c) / count(all) """
        for c in np.unique(self.y_train):
            self.prior[c] = np.sum(self.y_train == c) / self.train_size

    def findLikelihoodBernoulli(self):
        """ P(x|c) - Likelihood of feature given class P(x|c) = count(x|c) / count(c) """
        for feature in self.features:
            for c in np.unique(self.y_train):
                self.likelihood[feature][c] = np.sum(
                    self.X_train[self.y_train == c, feature]) / np.sum(self.y_train == c)
    def  findLikelihoodGaussian(self):
        
    def setBernoulliNaiveBayes(self):
        self.findClassPrior()
        self.findLikelihoodBernoulli()

    def setGaussianNaiveBayes(self):
        pass

    def fit(self, function, X, y):
        self.features = list(X.columns)
        self.X_train = X
        self.y_train = y
        self.train_size = X.shape[0]
        self.num_feats = X.shape[1]

        if function == 'Bernoulli':



IndentationError: expected an indented block after 'for' statement on line 7 (3008058220.py, line 9)

In [55]:
x = np.array([[1, 2, 2], [2, 1, 2], [1, 2, 1], [1, 1, 1]])
y = np.array([1, 2, 1, 2])
likelihood = {}
for feature in [1, 2, 3]:
    for c in np.unique(y):
        likelihood[feature][c] = np.sum(x[y == c, feature]) / np.sum(y == c)
print(likelihood)

KeyError: 1